# Data Science Ethics Checklist

[![Deon badge](https://img.shields.io/badge/ethics%20checklist-deon-brightgreen.svg?style=popout-square)](http://deon.drivendata.org/)

This notebook is for the Eviction Data Case Study exercise in the "Actionable Ethics for Data Scientists" workshop at ODSC. We will be walking through data and deon ethics checklist in the notebook. We will have a group discussion where there is a <span style="color:green">**\*\*bolded green discussion question\*\***</span>. Some code relevant to the checklist items are provided, but you are encouraged to think of other things to look at and share with the group.

## Background

Over the past five decades, [housing costs have risen faster than incomes](http://www.jchs.harvard.edu/state-nations-housing-2018), low-cost housing has been disappearing from the market, and racial disparities in homeownership rates have deepened. This has put many in a perilous situation. As the [Eviction Lab](https://evictionlab.org/why-eviction-matters/#affordable-housing-crisis) explains:

> Today, most poor renting families spend at least half of their income on housing costs, with one in four of those families spending over 70 percent of their income just on rent and utilities. Only one in four families who qualify for affordable housing programs get any kind of help. Under those conditions, it has become harder for low-income families to keep up with rent and utility costs, and a growing number are living one misstep or emergency away from eviction.


## Objective

A non-profit dedicated to helping people at risk of eviction in California has tasked us to build a model to estimate the number of eviction cases by geography, based on socioeconomic data. They would like to use these estimates to help them prioritize where to commit funding and resources.

We will be using a subset of the eviction dataset published by the [Eviction Lab](https://evictionlab.org/) at Princeton University. The subset is the census-tract-level aggregates for only tracts in the state of California. 

## Loading the Data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os.path
import pandas as pd
import seaborn as sns
import wget

%matplotlib inline
pd.set_option("display.max_columns", 30)

In [ ]:
DATA_URL = "https://drivendata-public-assets.s3.amazonaws.com/odsc-west-2019/california-tracts.csv"
DATA_PATH = "../data/raw/california-tracts.csv"

# Set standard fig size for plots
FIGSIZE = (10, 10)

In [ ]:
if not os.path.exists(DATA_PATH):
    # Download data
    wget.download(url=DATA_URL, out=DATA_PATH)

In [ ]:
df = pd.read_csv(DATA_PATH)
print(df.shape)
print(df.dtypes)
df.head()

In [ ]:
# count = non-NaN observations; size = all observations
df.groupby('year').agg(count = ('eviction-rate', 'count'), size = ('eviction-rate', 'size')).transpose()

In [ ]:
# Data Dictionary
DATA_DICT_URL = "https://drivendata-public-assets.s3.amazonaws.com/odsc-west-2019/DATA_DICTIONARY.txt"
DATA_DICT_PATH = "../references/DATA_DICTIONARY.txt"

if not os.path.exists(DATA_DICT_PATH):
    # Download data dictionary
    wget.download(url=DATA_DICT_URL, out=DATA_DICT_PATH)

In [ ]:
!cat $DATA_DICT_PATH

## A. Data Collection

- [ ] **A.1 Informed consent**: If there are human subjects, have they given informed consent, where subjects affirmatively opt-in and have a clear understanding of the data uses to which they consent?

We are not directly collecting any data from human subjects. We are using a well-known, publicly published dataset published by a well-known research laboratory at a prestigious research university. The university has strong data protection guidelines documented [here](https://ria.princeton.edu/research-data-security). Furthermore, the data represents public records such as court records and census data. 

 - [ ] **A.2 Collection bias**: Have we considered sources of bias that could be introduced during data collection and survey design and taken steps to mitigate those?

Data sources are reported by Eviction Lab [here](https://evictionlab.org/help-faq/#data-source). In their [methodology details](https://evictionlab.org/docs/Eviction%20Lab%20Methodology%20Report.pdf), they note that some observations are marked as "low", meaning that Eviction Lab believes they are undercounted based on comparisons against county-level aggregate data reported by states. 

There has also been some further criticism from housing activists that Eviction Lab is undercounting evictions when compared against data from local activist organizations, especially informal evictions that don't go through the formal legal process. See:

1. https://shelterforce.org/2018/08/22/eviction-lab-misses-the-mark/
2. https://chieforganizer.org/2018/08/28/the-cracks-in-the-eviction-lab-wall-are-undercounting-the-crisis/

 - [ ] **A.3 Limit PII exposure**: Have we considered ways to minimize exposure of personally identifiable information (PII), for example through anonymization or not collecting information that isn't relevant for analysis?

The dataset we are using is aggregated at the census tract level and does not directly have PII in it. The Eviction Lab has individual household data but does not make those available publicly. Again, the university has strong data protection guidelines documented [here](https://ria.princeton.edu/research-data-security).

More generally, one potential pitfall is that in situations where groups are very small, there is potential to infer or associate information to individuals. We can actually see in the data that some tracts have very small counts of renter households. However, we don't have any PII-relevant fields in this dataset, and identification in this case would involve joining to other datasets that could independently be used to identify people. 

In [ ]:
df['renter-occupied-households'].describe(percentiles=[0.001, 0.01, 0.05, 0.10])

## B. Data Storage

 - [ ] **B.1 Data security**: Do we have a plan to protect and secure data (e.g., encryption at rest and in transit, access controls on internal users and third parties, access logs, and up-to-date software)?

Since we are using a public dataset, this is not applicable.

 - [ ] **B.2 Right to be forgotten**: Do we have a mechanism through which an individual can request their personal information be removed?

Since we are using a public dataset and not collecting data from subjects directly, this is not applicable.

 - [ ] **B.3 Data retention plan**: Is there a schedule or plan to delete the data after it is no longer needed?

Since we are using a public dataset, this is not applicable.

## C. Analysis

 - [ ] **C.1 Missing perspectives**: Have we sought to address blindspots in the analysis through engagement with relevant stakeholders (e.g., checking assumptions and discussing implications with affected communities and subject matter experts)?

As noted in A.2, there may be systemic undercounting of evictions due to the data collection process. In particular, for informal evictions that bypassed the formal legal process, there may be certain groups of people that an analysis based on this dataset is completely blind to. 

 - [ ] **C.2 Dataset bias**: Have we examined the data for possible sources of bias and taken steps to mitigate or address these biases (e.g., stereotype perpetuation, confirmation bias, imbalanced classes, or omitted confounding variables)?

<span style="color:green">**\*\*Discussion: What are possible sources of bias? Take some time to explore the data.\*\***</span> 

In [ ]:
print('Proportion of observations with low-flag:', df['low-flag'].mean())
print('Count of observations with low-flag')
df['low-flag'].value_counts()

In [ ]:
print('Proportion of observations with imputed flag:', df['imputed'].mean())
print('Count of observations with imputed flag')
df['imputed'].value_counts()

In [ ]:
df['evictions_is_na'] = df['evictions'].isna()
print('Evictions percent NA: ', df['evictions_is_na'].mean())
df['evictions_is_na'].value_counts()

In [ ]:
# Helper function to expand y axis for plots
def expand_ylim():
    # Seaborn cuts off top and bottom with matplotlib 3.1.1, so we need to add padding.
    b, t = plt.ylim() # get values for bottom and top
    b += 0.5 # Add 0.5 to the bottom
    t -= 0.5 # Subtract 0.5 from the top
    plt.ylim(b, t) # reset ylim values

In [ ]:
# Correlation heatmap
corr = df.loc[:, [col for col in df.columns if col not in ('GEOID', 'name')]].corr()
plt.figure(figsize=FIGSIZE)
sns.heatmap(corr, 
            vmin=-1.0,
            vmax=1.0,
            cmap='RdBu_r');

expand_ylim()
plt.show()

In [ ]:
race_cols = ['pct-white', 'pct-af-am', 'pct-hispanic', 'pct-am-ind', 
        'pct-asian', 'pct-nh-pi', 'pct-multiple', 'pct-other']
df[race_cols].describe(percentiles=[])

In [ ]:
# Calculate pairwise correlation of some column against race percentage columns
col_to_correlate = 'low-flag'
race_cols = ['pct-white', 'pct-af-am', 'pct-hispanic', 'pct-am-ind', 
        'pct-asian', 'pct-nh-pi', 'pct-multiple', 'pct-other']
df[race_cols + [col_to_correlate]].corr()[col_to_correlate][:-1]

In [ ]:
# Visualize the above correlations.
cols_to_correlate = ['evictions', 'median-household-income']
heatmap_df = pd.DataFrame(df[race_cols + cols_to_correlate].corr()[cols_to_correlate][:-2])

plt.figure(figsize=FIGSIZE)
sns.heatmap(heatmap_df.sort_values('evictions'), annot=True, fmt="g", cmap='RdBu_r', vmin=-.5, vmax=.5)

expand_ylim()

 - [ ] **C.3 Honest representation**: Are our visualizations, summary statistics, and reports designed to honestly represent the underlying data?

We are not going to spend time in this exercise looking at examples of poor visualizations, but this is a good check to do for real projects.

 - [ ] **C.4 Privacy in analysis**: Have we ensured that data with PII are not used or displayed unless necessary for the analysis?

Not applicable: We have no PII, but some small counties. We need to be mindful of those and maybe exclude them from visualizations or combine them with neighboring tracts.

 - [ ] **C.5 Auditability**: Is the process of generating the analysis well documented and reproducible if we discover issues in the future?

Yes, all of our code is contained in this notebook. For real projects, this is an important point to consider in more depth.

## D. Modeling

Though discrimination may not be strictly a problem in this situation, the non-profit wants their decision-making to be race-blind, so they ask for the population race percentage features to not be included in the modeling.

In [ ]:
TARGET_VAR = 'evictions'
FEATURE_VARS = [
    'year', 'population', 'poverty-rate', 'median-property-value',
    'renter-occupied-households', 'pct-renter-occupied', 
    'median-gross-rent', 'median-household-income',  'rent-burden', 
    
    # Don't include race features
    #'pct-white' , 'pct-af-am', 'pct-hispanic', 'pct-am-ind', 
    #'pct-asian', 'pct-nh-pi', 'pct-multiple', 'pct-other'
]
GROUP_VAR = 'GEOID' # Prevent leakage

In [ ]:
print(f"Original Shape: {df.shape}")
# Drop NAs in target variable
df_modeling = df.dropna(subset=[TARGET_VAR])
df_modeling.reset_index(inplace=True)
print(f"Shape without NAs: {df_modeling.shape}")

In [ ]:
from sklearn.model_selection import GroupShuffleSplit, cross_validate
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [ ]:
df_modeling.loc[:, TARGET_VAR].describe()

In [ ]:
split = GroupShuffleSplit(
    test_size=0.20, 
    n_splits=2, 
    random_state=36
).split(df_modeling, groups=df_modeling.loc[:, GROUP_VAR])

train_inds, test_inds = next(split)

df_train = df_modeling.loc[train_inds, :]
df_test = df_modeling.loc[test_inds, :]

X_train = df_train.loc[:, FEATURE_VARS].values
y_train = df_train.loc[:, TARGET_VAR].values

X_test = df_test.loc[:, FEATURE_VARS].values
y_test = df_test.loc[:, TARGET_VAR].values

In [ ]:
%%time
model_pipeline = Pipeline([
    ('med_impute', SimpleImputer(strategy='median')),
    ('model', RandomForestRegressor(
        criterion='mse',
        n_estimators=100, 
        max_depth=10,
        random_state=36
    ))
])
model_pipeline.fit(X_train, y_train)

In [ ]:
y_pred = model_pipeline.predict(X_test)
print("R2", r2_score(y_test, y_pred))
print("MSE", mean_squared_error(y_test, y_pred))
print("MAE", mean_absolute_error(y_test, y_pred))

 - [ ] **D.1 Proxy discrimination**: Have we ensured that the model does not rely on variables or proxies for variables that are unfairly discriminatory?

As noted, unfair discrimination is not directly an issue with this use case, but the non-profit wants the model not to factor in race. 


<span style="color:green">**\*\*Discussion: Are there concerns about proxy variables for race? Take some time to explore the data.\*\***</span> 

In [ ]:
race_vars = ['pct-white' , 'pct-af-am', 'pct-hispanic', 'pct-am-ind', 
             'pct-asian', 'pct-nh-pi', 'pct-multiple', 'pct-other']
corr = df_modeling.loc[:, FEATURE_VARS + race_vars].corr().loc[FEATURE_VARS, race_vars]

plt.figure(figsize=FIGSIZE)
sns.heatmap(corr, 
            vmin=-1.0,
            vmax=1.0,
            cmap='RdBu_r'
           )

expand_ylim()

 - [ ] **D.2 Fairness across groups**: Have we tested model results for fairness with respect to different affected groups (e.g., tested for disparate error rates)?

<span style="color:green">**\*\*Discussion: Are there concerns about fairness across groups? Take some time to explore the data.\*\***</span> 

In [ ]:
# Calculate correlation of prediction error with race
y_error = y_pred - y_test

df_test['error'] = y_error
error_corr = df_test[race_vars + ['error']].corr()['error'][:-1]

df_test['abs_error'] = abs(y_error)
abs_error_corr = df_test[race_vars + ['abs_error']].corr()['abs_error'][:-1]

pd.DataFrame({'corr_with_error': error_corr, 'corr_with_abs_error': abs_error_corr})

In [ ]:
# Produce scatter plots of error for predominantly hispanic tracts
df_test['true'] = y_test
df_test['pred'] = y_pred

hisp = df_test[df_test['pct-hispanic'] > 75]
hisp = hisp.sort_values('evictions')

plt.scatter(np.arange(len(hisp)), hisp['pred'], s=1)
plt.scatter(np.arange(len(hisp)), hisp['true'], s=1);

In [ ]:
# Produce same plot side by side for different populations
fig, axes = plt.subplots(1, 3)
fig.set_figwidth(20)
fig.suptitle('Predicted Evictions (blue) vs. Actual (orange) for Tracts by Majority Race', 
             fontsize=16,
             y=1.1)

# Abstract plotting into helper function to plot the error for each population
def plot_error_for_pop(pop_name, ax, thresh=50):
    pop = df_test[df_test[pop_name] > thresh]
    pop = pop.sort_values('evictions')
    ax.scatter(np.arange(len(pop)), pop['pred'], s=1)
    ax.scatter(np.arange(len(pop)), pop['true'], s=1)
    ax.set_ylim(0, 100)
    ax.set_title(f"> {thresh} {pop_name}")

for ax, pop_name in zip(axes, ['pct-af-am', 'pct-hispanic', 'pct-white']):
    plot_error_for_pop(pop_name, ax)

 - [ ] **D.3 Metric selection**: Have we considered the effects of optimizing for our defined metrics and considered additional metrics?

In the Random Forest model above, we used `mse` (mean-squared error) as the loss function. Mean-squared error is affected more by outliers than mean-absolute error. 

 - [ ] **D.4 Explainability**: Can we explain in understandable terms a decision the model made in cases where a justification is needed?

<span style="color:green">**\*\*Discussion: Are there concerns about model explainability? Take some time to explore the data.\*\***</span> 

In [ ]:
model_pipeline.named_steps['model'].feature_importances_
feature_importance = pd.DataFrame.from_dict({
    'features': FEATURE_VARS,
    'importance': model_pipeline.named_steps['model'].feature_importances_
})
print(feature_importance.sort_values('importance', ascending=False).reset_index(drop=True))

 - [ ] **D.5 Communicate bias**: Have we communicated the shortcomings, limitations, and biases of the model to relevant stakeholders in ways that can be generally understood?

<span style="color:green">**\*\*Discussion: What are shortcomings or limitations of our model we should be concerned about?.\*\***</span> 

## E. Deployment

 - [ ] **E.1 Redress**: Have we discussed with our organization a plan for response if users are harmed by the results (e.g., how does the data science team evaluate these cases and update analysis and models to prevent future harm)?

This is important to consider in the context of how analysis and model results are used. Are the results used to make positive or negative interventions? What is the potential harm or inequity from incorrect model estimates?

 - [ ] **E.2 Roll back**: Is there a way to turn off or roll back the model in production if necessary?

 - [ ] **E.3 Concept drift**: Do we test and monitor for concept drift to ensure the model remains fair over time?

 - [ ] **E.4 Unintended use**: Have we taken steps to identify and prevent unintended uses and abuse of the model and do we have a plan to monitor these once the model is deployed?

*Data Science Ethics Checklist generated with [deon](http://deon.drivendata.org).*